## HIV News articles extraction from VietNam News. Data Extraction of following parameters
- Headline
- Description
- Author
- Published Date
- Category
- Publication
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions
from dateutil import parser # date formet handling

### Creating Empty lists for HIV News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls, category, publication = [], [], [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'HIV site:vietnamnews.vn'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

soup = BeautifulSoup(get(url).text, 'lxml')
try:
    # Extracts the digits if it the resulted number without comma ','. eg: About 680 results (0.23 seconds)
    max_pages = round([int(s) for s in soup.select_one('div#resultStats').text.split() if s.isdigit()][0]/10)
    max_pages = max_pages + 1
except:
    # Extracts the digits if it the resulted number without comma ','. eg: About 1,080 results (0.23 seconds)
    max_pages = round(int(''.join(i for i in soup.select_one('div#resultStats').text if i.isdigit()))/10)
    max_pages = max_pages + 1

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print("Total Extracted URL's are" + ' : ' + str(len(urls)), type(urls))

Total Extracted URL's are : 292 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            headlines.append(article.title.strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            descriptions.append(' '.join(article.meta_data['og']['description'].split()))
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            authors.append(article.authors)
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            dates.append(str(parser.parse(soup.select_one('.vnnews-time-post').span.text.strip())).split()[0])
        except:
            dates.append(None)
            
        # Extracts the news category
        try:
            category.append(soup.select_one('.cate-act').text)
        except:
            category.append(None)
            
        # Extracts the news articles
        try:
            news.append(' '.join(article.text.split()).replace("\'\'"," ").replace("\'", "").replace(" / ", ""))
        except:
            news.append(None)
            
        # Extracts the news publication
        try:
            publication.append(article.meta_data['og']['site_name'])
        except:
            publication.append(None)

        # Extracts Keywords and Summaries
        try:
            keywords.append(article.keywords)
            summaries.append(' '.join(article.summary.split()))
        except:
            keywords.append(None)
            summaries.append(None)
                        
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        category.append(None)
        publication.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

You must `download()` an article first!75/doctors-nurses-exposed-to-hiv-in-emergency-surgery.htmliv-aids.html017-in-hcm-city.htmltmlml
You must `download()` an article first!258/regional-experts-share-hiv-monitoring-experience.html-aids-drugs.html
You must `download()` an article first!591/children-with-hiv-celebrate-mid-autumn-fest.html
You must `download()` an article first!expat-corner/374148/vn-american-swaps-body-building-for-community-building.htmlovincial-youth.html
You must `download()` an article first!862/healthcare-sector-strives-to-control-epidemic.html
You must `download()` an article first!vaac.htmliefs-23-01-2006.htmly.htmlids-fight.htmll-in-da-nang.htmltml
Wall time: 59min 38snews.vn/society/owners-of-the-future/188808/street-kids-overjoyed-at-free-medical-treatment.html


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(category), len(publication), len(news), len(keywords), len(summaries), len(urls))

292 292 292 292 292 292 292 292 292 292


### Creating a csv file after checking array length and droping the missing values from the dataset

In [8]:
if len(headlines) == len(descriptions) == len(authors) == len(dates) == len(news) == len(publication) == len(keywords) == len(summaries) == len(urls) == len(category):
    tbl = pd.DataFrame({'Headlines' : headlines,
                        'Descriptions' : descriptions,
                        'Authors' : authors,
                        'Published_Dates' : dates,
                        'Publication' : publication,
                        'Articles' : news,
                        'category' : category,
                        'Keywords' : keywords,
                        'Summaries' : summaries,
                        'Source_URLs' : urls})
    tbl.dropna()
    path = 'D:\\#Backups\\Desktop\\!Code!\\CDRI\\HIV\\Data Extraction\\#Datasets\\'
    tbl.to_csv(path+'VietNam News.csv', index=False)
else:
    print('Array lenght does not match!')
    
tbl.head()

,Headlines,Descriptions,Authors,Published_Dates,Publication,Articles,category,Keywords,Summaries,Source_URLs
0,Number of HIV/AIDS cases continues to fall,The number of HIV/AIDS cases in Việt Nam conti...,[],2018-12-03,vietnamnews.vn,HCM CITY– The number of HIV/AIDS cases in Việt...,Society,"[cases, number, prep, services, việt, nam, fal...",HCM CITY– The number of HIV/AIDS cases in Việt...,http://vietnamnews.vn/society/481242/number-of...
1,HIV increase sparks free tests,The Ministry of Health has asked the northern ...,[],2018-08-17,vietnamnews.vn,A delegation from the health ministry visits r...,Society,"[ministry, thượng, patients, hiv, hivaids, spa...",A delegation from the health ministry visits r...,http://vietnamnews.vn/society/463991/hiv-incre...
2,HIV testing and treatment for an AIDS-free world,The Human Immunodeficiency Virus (HIV) is spre...,[],2018-06-25,vietnamnews.vn,By Dr Mathieu Nalpas* The Human Immunodeficien...,Life & Style,"[testing, medical, status, việt, living, hiv, ...",By Dr Mathieu Nalpas*The Human Immunodeficienc...,http://vietnamnews.vn/life-style/450345/hiv-te...
3,Localities to strengthen HIV/AIDS control and ...,The Ministry of Health has asked provinces and...,[],2018-10-30,vietnamnews.vn,HÀ NỘI — The Ministry of Health has asked prov...,Society,"[ministry, localities, strengthen, living, hiv...",HÀ NỘI — The Ministry of Health has asked prov...,http://vietnamnews.vn/society/468834/localitie...
4,VN has work to do in HIV prevention and treatment,"About 50,000 people living with HIV in Việt Na...",[],2018-11-17,vietnamnews.vn,"HÀ NỘI — About 50,000 people living with HIV i...",Society,"[patients, việt, vn, hiv, nam, living, hivaids...","HÀ NỘI — About 50,000 people living with HIV i...",http://vietnamnews.vn/society/480271/vn-has-wo...


In [9]:
tbl.shape

(292, 10)